In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as funcs
from pyspark.sql import SQLContext
from pyspark import Row
import os

In [2]:
os.environ["SPARK_HOME"] = "/usr/hdp/current/spark-client"
conf = (SparkConf().setMaster('local').setAppName('a'))

In [3]:
sc = SparkContext()
sqlContext = SQLContext(sc)

In [4]:

df = sqlContext.read.parquet("/tdg/2017/04/11/aggregated_metrics_hessen/directions/tiles/tile_direction_hours")

In [5]:
df.where("tile_id = 3787192").show()

+-------+--------+--------+---------+-------+--------+--------+---------+-------+--------+--------------+---------------+-------------+--------------+--------------+---------------+-------------+--------------+
|tile_id|ts_start|north_in|north_out|east_in|east_out|south_in|south_out|west_in|west_out|north_in_speed|north_out_speed|east_in_speed|east_out_speed|south_in_speed|south_out_speed|west_in_speed|west_out_speed|
+-------+--------+--------+---------+-------+--------+--------+---------+-------+--------+--------------+---------------+-------------+--------------+--------------+---------------+-------------+--------------+
+-------+--------+--------+---------+-------+--------+--------+---------+-------+--------+--------------+---------------+-------------+--------------+--------------+---------------+-------------+--------------+



In [6]:
df = df.withColumn("in_total", df.north_in + df.east_in + df.south_in + df.west_in)
df = df.withColumn("out_total", df.north_out + df.east_out + df.south_out + df.west_out)

In [7]:
df = df.withColumn("diff", df.in_total - df.out_total)

In [8]:
df.select("tile_id", "diff", "in_total", "out_total").orderBy("diff").show(10, False)

+-------+------+--------+---------+
|tile_id|diff  |in_total|out_total|
+-------+------+--------+---------+
|1945745|-23280|0       |23280    |
|1945745|-22280|0       |22280    |
|1945745|-21859|0       |21859    |
|1945745|-21729|0       |21729    |
|1945745|-20080|0       |20080    |
|1945745|-20057|0       |20057    |
|1945745|-20047|0       |20047    |
|1945745|-19654|0       |19654    |
|1945745|-18517|0       |18517    |
|1945745|-18377|0       |18377    |
+-------+------+--------+---------+
only showing top 10 rows



In [9]:
import pandas as pd
import numpy as np
from pyspark.sql.functions import col
tileL = pd.read_csv("tileList.csv")
t = np.unique(tileL['tile_id'])
ddf = df.where(col('tile_id').isin(list(t)))
ddf.show()


+-------+--------------------+--------+---------+-------+--------+--------+---------+-------+--------+------------------+------------------+------------------+-----------------+--------------+---------------+-----------------+-----------------+--------+---------+-----+
|tile_id|            ts_start|north_in|north_out|east_in|east_out|south_in|south_out|west_in|west_out|    north_in_speed|   north_out_speed|     east_in_speed|   east_out_speed|south_in_speed|south_out_speed|    west_in_speed|   west_out_speed|in_total|out_total| diff|
+-------+--------------------+--------+---------+-------+--------+--------+---------+-------+--------+------------------+------------------+------------------+-----------------+--------------+---------------+-----------------+-----------------+--------+---------+-----+
|5656465|2017-04-11T06:00:...|       0|     1242|      0|       0|       0|        0|      0|       0|               0.0|114.20132894246451|               0.0|              0.0|           0.

In [18]:
#ddf.repartition(1).write.format("com.databricks.spark.csv").save("tileCounts.csv",header=True)
from pyspark.sql.types import *
from pyspark.sql.functions import col, udf
ddf.coalesce(1).write.mode("overwrite").format('com.databricks.spark.csv').save("tileCounts.csv",header=True)

Py4JError: An error occurred while calling o1541.option. Trace:
py4j.Py4JException: Method option([class java.lang.String, class java.lang.Boolean]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:335)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:344)
	at py4j.Gateway.invoke(Gateway.java:252)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:133)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:209)
	at java.lang.Thread.run(Thread.java:745)



In [10]:
zipD = ddf.toPandas()
zipD.to_csv("tileCounts.csv",index=False)